Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License

# Distributed Training For Extractive Summarization on CNN/DM Dataset

## Summary
This notebook demonstrates how to use Azure Machine Learning to run distributed training using Distributed Data Parallel in Pytorch for extractive summarization. For more detailed model related information, please see [extractive_summarization_cnndm_transformer.ipynb](extractive_summarization_cnndm_transformer.ipynb)

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, refer to the [Configuration Notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) first if you haven't already to establish your connection to the AzureML Workspace. Prerequisites are:

- Azure subscription
- Azure Machine Learning Workspace
- Azure Machine Learning SDK

To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](summarization_evaluation.ipynb). 

## Create AML Workspace 

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath('../../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)
    
from utils_nlp.azureml import azureml_utils

# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

from azureml.core import Experiment, Workspace, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import PyTorch
from azureml.widgets import RunDetails
from azureml.train.dnn import Nccl

Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core:PipelineRun._from_dto with exception (azureml-core 1.0.83 (/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages), Requirement.parse('azureml-core==1.0.57.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core:StepRun._from_reused_dto with exception (azureml-core 1.0.83 (/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages), Requirement.parse('azureml-core==1.0.57.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core:StepRun._from_dto with exception (azureml-core 1.0.83 (/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages), Requirement.parse('azureml-core==1.0.57.*')).


SDK version: 1.0.83


In [4]:
## Replace the following constants with your own 
WORKSPACE_NAME = "daden1amlws"
SUBSRIPTION_ID = "9086b59a-02d7-4687-b3fd-e39fa5e0fd9b" 
RESOURCE_GROUP = "daden1aml"
LOCATION = "eastus2"

In [5]:
# Create the workspace using the specified parameters
ws = Workspace.create(name = WORKSPACE_NAME,
                      subscription_id = SUBSRIPTION_ID,
                      resource_group = RESOURCE_GROUP, 
                      location = LOCATION,
                      create_resource_group = False,
                      exist_ok = True)
ws.get_details()

# write the details of the workspace to a configuration file to the notebook library
ws.write_config()

In [6]:
# Retrieve the workspace
ws = Workspace.setup()

# Print the workspace attributes
print('Workspace name: ' + ws.name, 
      'Workspace region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

setup() is now deprecated. Instead, please use create() to create a new workspace, or get()/from_config() to retrieve an existing one


Workspace name: daden1amlws
Workspace region: eastus2
Subscription id: 9086b59a-02d7-4687-b3fd-e39fa5e0fd9b
Resource group: daden1aml


## Create Compute Cluster

In [7]:
## Replace the following constants with your own 
AMLCOMPUTE_CLUSTER_NAME = "extsum5"
NODE_COUNT = 4
VM_SIZE = 'STANDARD_NC6'

In [8]:
try:
    gpu_compute_target = ComputeTarget(workspace=ws, name=AMLCOMPUTE_CLUSTER_NAME)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size=VM_SIZE,
                                                           max_nodes=NODE_COUNT,
                                                           NodeIdleTimeBeforeScaleDown='PT1200S')

    # create the cluster
    gpu_compute_target = ComputeTarget.create(ws, AMLCOMPUTE_CLUSTER_NAME, compute_config)

    gpu_compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute. 
print(gpu_compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-01-30T22:43:39.856000+00:00', 'errors': None, 'creationTime': '2020-01-23T04:50:26.160743+00:00', 'modifiedTime': '2020-01-23T20:31:35.349184+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1200S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


## Create Experiment

In [9]:
## Replace the following constants with your own 
EXPERIMENT_NAME = 'NLP-ExtSum'

In [10]:
experiment = Experiment(ws, name=EXPERIMENT_NAME)

## Download dataset to local file system

In [11]:
## local folder to save the downloaded data
LOCAL_DATA_FOLDER = '/dadendev/bertsumdata/'

In [12]:
!mkdir -p {LOCAL_DATA_FOLDER}

In [ ]:
from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarizationDataset
CNNDMBertSumProcessedData.download(local_path=LOCAL_DATA_FOLDER)

## Upload the downloaded dataset to AML workspace

In [14]:
## folder in the workspace where the data is uploaded to
TARGET_DATA_FOLDER = '/bertsum_processed_data'

In [15]:
ds = ws.get_default_datastore()

In [16]:
ds.upload(src_dir=LOCAL_DATA_FOLDER, target_path=TARGET_DATA_FOLDER)

## Prepare the local project folder which is mirror to the workspace for the experiment

In [17]:
## local folder to store all the related files to be copied to the workspace
PROJECT_FOLDER = './azureml_exp'
## conda environment name, the yaml file will be copied to the workspace
CONDA_ENV_NAME = "nlp_gpu"

In [18]:
ENTRY_SCRIPT = "extractive_summarization_cnndm_distributed_train.py"

In [19]:
!mkdir -p {PROJECT_FOLDER}
!python ../../tools/generate_conda_file.py --gpu --name {CONDA_ENV_NAME}
!cp ./nlp_gpu.yaml {PROJECT_FOLDER}
!cp {ENTRY_SCRIPT} {PROJECT_FOLDER}
!cp -r ../../utils_nlp {PROJECT_FOLDER}


Generated conda file: nlp_gpu.yaml

To create the conda environment:
$ conda env create -f nlp_gpu.yaml

To update the conda environment:
$ conda env update -f nlp_gpu.yaml

To register the conda environment in Jupyter:
$ conda activate nlp_gpu
$ python -m ipykernel install --user --name nlp_gpu --display-name "Python (nlp_gpu)"



## Submit Run

In [20]:
#AZUREML_CONFIG_PATH = "./.azureml"
## output dir in the workspace
MODEL_NAME = "distilbert-base-uncased"
ENCODER = "transformer"
TARGET_OUTPUT_DIR = f'output/{EXPERIMENT_NAME}/'
## cache dir in the workspace
TARGET_CACHE_DIR = f'cache/{EXPERIMENT_NAME}/'
## file name for saving the prediction
SUMMARY_FILENAME = "generated_summaries.txt"
MODEL_FILENAME = "dist_extsum.pt"

## local path to download the output from the cluster
LOCAL_OUTPUT_DIR = './output'

In [21]:
os.makedirs(LOCAL_OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(LOCAL_OUTPUT_DIR, EXPERIMENT_NAME), exist_ok=True)

In [22]:
NcclConfig=Nccl()
estimator = PyTorch(source_directory=PROJECT_FOLDER,
                    compute_target=gpu_compute_target,
                    script_params={
                        "--dist_url": "$AZ_BATCHAI_PYTORCH_INIT_METHOD",
                        "--rank": "$AZ_BATCHAI_TASK_INDEX",
                        "--node_count": NODE_COUNT,
                        "--data_dir":ds.path(f'{TARGET_DATA_FOLDER}').as_mount(),
                        "--cache_dir": ds.path(f'{TARGET_CACHE_DIR}').as_mount(),
                        '--output_dir':ds.path(f'{TARGET_OUTPUT_DIR}').as_mount(),
                        "--quick_run": 'false',
                        "--summary_filename": f'{SUMMARY_FILENAME}',
                        "--model_filename": f'{MODEL_FILENAME}',
                        "--model_name": MODEL_NAME,
                        "--encoder": ENCODER
                    },
                    entry_script= ENTRY_SCRIPT,
                    node_count=NODE_COUNT,
                    distributed_training=NcclConfig,
                    conda_dependencies_file=f'{CONDA_ENV_NAME}.yaml',
                    use_gpu=True)

If environment_definition or conda_dependencies_file is specified, Azure ML will not install any framework related packages on behalf of the user.
framework_version is not specified, defaulting to version 1.3.


In [23]:
run = experiment.submit(estimator)

In [24]:

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
"""
If you stop the notebook and come back, 
you'll need to use the run_id in the output of the previous cell 
to get run details.
"""
# fetched_run = Run(experiment, "NLP-ExtSum_1579816237_ea238f69")
# RunDetails(fetched_run).show()

## Download Generated Summaries 

In [25]:
# need to clear the local output dir as the ds.download worn't download is the path exists
!rm -rf {LOCAL_OUTPUT_DIR}/* 

In [32]:
ds.download(target_path=LOCAL_OUTPUT_DIR,
                   prefix=f'{TARGET_OUTPUT_DIR}{SUMMARY_FILENAME}',
                   show_progress=True)

Downloaded output/NLP-ExtSum/generated_summaries.txt, 1 files out of an estimated total of 1


1

## Evaluation

In [27]:
from utils_nlp.eval.evaluate_summarization import get_rouge
from utils_nlp.models.transformers.extractive_summarization import ExtSumProcessedData
import pickle
from utils_nlp.models.transformers.extractive_summarization import ExtractiveSummarizer

In [28]:
train_dataset, test_dataset = ExtSumProcessedData().splits(root=LOCAL_DATA_FOLDER)

In [30]:
target = [i['tgt_txt'] for i in test_dataset]

In [33]:
prediction = []
with open(os.path.join(LOCAL_OUTPUT_DIR, f'{TARGET_OUTPUT_DIR}{SUMMARY_FILENAME}'), "r") as filehandle:
    for cnt, line in enumerate(filehandle):
        prediction.append(line[0:-1]) # remove the ending "\n"

In [34]:
## you can also download the saved model and run prediction if you are running the notebook on a gpu machine
"""
ds.download(target_path=LOCAL_OUTPUT_DIR,
               prefix=f'{TARGET_OUTPUT_DIR}{MODEL_FILENAME}',
               show_progress=True)
summarizer = ExtractiveSummarizer(MODEL_NAME, ENCODER, LOCAL_OUTPUT_DIR)
summarizer.model.load_state_dict(
    torch.load(os.path.join(LOCAL_OUTPUT_DIR, f'{TARGET_OUTPUT_DIR}{MODEL_FILENAME}'))
)
prediction = summarizer.predict(test_dataset, num_gpus=torch.cuda.device_count(), batch_size=128)
"""

Downloaded output/NLP-ExtSum/dist_extsum.pt, 1 files out of an estimated total of 1


Scoring: 100%|██████████| 90/90 [00:41<00:00,  3.70it/s]


In [35]:
test_dataset[0]['src_txt']

['a university of iowa student has died nearly three months after a fall in rome in a suspected robbery attack in rome .',
 'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program in italy when the incident happened in january .',
 'he was flown back to chicago via air ambulance on march 20 , but he died on sunday .',
 'andrew mogni , 20 , from glen ellyn , illinois , a university of iowa student has died nearly three months after a fall in rome in a suspected robbery',
 'he was taken to a medical facility in the chicago area , close to his family home in glen ellyn .',
 "he died on sunday at northwestern memorial hospital - medical examiner 's office spokesman frank shuftan says a cause of death wo n't be released until monday at the earliest .",
 'initial police reports indicated the fall was an accident but authorities are investigating the possibility that mogni was robbed .',
 "on sunday , his cousin abby wrote online : ` this morning my cous

In [36]:
prediction[0]

'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program in italy when the incident happened in january .<q>a university of iowa student has died nearly three months after a fall in rome in a suspected robbery attack in rome .<q>he was flown back to chicago via air ambulance on march 20 , but he died on sunday .'

In [37]:
target[0]

'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program when the incident happened in january<q>he was flown back to chicago via air on march 20 but he died on sunday<q>initial police reports indicated the fall was an accident but authorities are investigating the possibility that mogni was robbed<q>his cousin claims he was attacked and thrown 40ft from a bridge'

In [38]:
RESULT_DIR = TemporaryDirectory().name

In [40]:
rouge_score = get_rouge(prediction, target, RESULT_DIR)

11489
11489


2020-01-31 03:38:44,480 [MainThread  ] [INFO ]  Writing summaries.
2020-01-31 03:38:44,481 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmpz9fasy6o/tmpnhtvedhc/system and model files to /tmp/tmpz9fasy6o/tmpnhtvedhc/model.
2020-01-31 03:38:44,481 [MainThread  ] [INFO ]  Processing files in /tmp/tmpz9fasy6o/rouge-tmp-2020-01-31-03-38-43/candidate/.
2020-01-31 03:38:45,679 [MainThread  ] [INFO ]  Saved processed files to /tmp/tmpz9fasy6o/tmpnhtvedhc/system.
2020-01-31 03:38:45,681 [MainThread  ] [INFO ]  Processing files in /tmp/tmpz9fasy6o/rouge-tmp-2020-01-31-03-38-43/reference/.
2020-01-31 03:38:46,904 [MainThread  ] [INFO ]  Saved processed files to /tmp/tmpz9fasy6o/tmpnhtvedhc/model.
2020-01-31 03:38:46,989 [MainThread  ] [INFO ]  Written ROUGE configuration to /tmp/tmpz9fasy6o/tmpicwmu3se/rouge_conf.xml
2020-01-31 03:38:46,989 [MainThread  ] [INFO ]  Running ROUGE with command /dadendev/pyrouge/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl -e /dadendev/pyrouge/tools/

---------------------------------------------
1 ROUGE-1 Average_R: 0.52322 (95%-conf.int. 0.52036 - 0.52598)
1 ROUGE-1 Average_P: 0.35403 (95%-conf.int. 0.35171 - 0.35628)
1 ROUGE-1 Average_F: 0.40840 (95%-conf.int. 0.40623 - 0.41040)
---------------------------------------------
1 ROUGE-2 Average_R: 0.23066 (95%-conf.int. 0.22789 - 0.23341)
1 ROUGE-2 Average_P: 0.15558 (95%-conf.int. 0.15365 - 0.15744)
1 ROUGE-2 Average_F: 0.17947 (95%-conf.int. 0.17740 - 0.18147)
---------------------------------------------
1 ROUGE-L Average_R: 0.47554 (95%-conf.int. 0.47274 - 0.47834)
1 ROUGE-L Average_P: 0.32224 (95%-conf.int. 0.32001 - 0.32440)
1 ROUGE-L Average_F: 0.37150 (95%-conf.int. 0.36935 - 0.37361)

